In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
driver = uc.Chrome()
driver.get("https://search.naver.com/search.naver?ssc=tab.news.all&query=IM%EB%B1%85%ED%81%AC&sm=tab_opt&sort=0&photo=0&field=0&pd=5&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3A1y&is_sug_officeid=0&office_category=&service_area=")

In [19]:
box = []
while True:
    driver.find_element(By.CSS_SELECTOR, "ul.list_news._infinite_list").find_elements(By.CSS_SELECTOR, "span.sds-comps-text.sds-comps-text-type-body2.sds-comps-text-weight-sm")
    for i in news:
        title = i.find_element(By.CSS_SELECTOR, "h2#title_area span").text
        summary = i.find_element(By.CSS_SELECTOR, "strong.media_end_summary").text
        A = [title, summary]
        if A not in box:
            box.append(A)
            if len(box) == 100:
                break
    if len(box) == 100:
        break
    print(f"{len(box)}개 뉴스기사 수집 완료")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

[<undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277bafce71f", element="f.2ED07E3901BF288CD8EB32A8B52A9ABC.d.04A0235BAF5ACCB571B325376538E800.e.1510")>,
 <undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277bafce71f", element="f.2ED07E3901BF288CD8EB32A8B52A9ABC.d.04A0235BAF5ACCB571B325376538E800.e.1512")>,
 <undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277bafce71f", element="f.2ED07E3901BF288CD8EB32A8B52A9ABC.d.04A0235BAF5ACCB571B325376538E800.e.1516")>,
 <undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277bafce71f", element="f.2ED07E3901BF288CD8EB32A8B52A9ABC.d.04A0235BAF5ACCB571B325376538E800.e.1517")>,
 <undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277bafce71f", element="f.2ED07E3901BF288CD8EB32A8B52A9ABC.d.04A0235BAF5ACCB571B325376538E800.e.1519")>,
 <undetected_chromedriver.webelement.WebElement (session="f42572c29194d3dce0547277baf

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--start-maximized")

# ChromeDriver 경로 설정 (직접 다운로드한 위치로 수정)
service = Service(executable_path="C:/경로/chromedriver.exe")

# 드라이버 실행 테스트
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://naver.com")


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [13]:
import requests
import urllib.parse
import time

def crawl_naver_news(query="IM뱅크", total=100):
    encoded_query = urllib.parse.quote(query)
    base_referer = f'https://search.naver.com/search.naver?ssc=tab.news.all&query={encoded_query}'

    headers = {
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'origin': 'https://search.naver.com',
        'referer': base_referer,
        'user-agent': 'Mozilla/5.0',
    }

    cookies = {
        'NNB': '65XOKTYQZJDWQ',
        'BUC': 'N4TT_tBwMK7oN8LsdlB7Jx9ASUG8r9f0w-j9-Lk1Heg=',
    }

    news_list = []

    for start in range(1, total + 1, 10):
        params = {
            'query': query,
            'nso': 'so:r,p:1y,a:all',
            'pd': '5',
            'sort': '0',
            'start': str(start),
            'ssc': 'tab.news.all',
            'sm': 'tab_smr',
            'photo': '0',
            'field': '0',
            'office_type': '0',
            'office_section_code': '0',
            'mynews': '0',
        }

        url = "https://s.search.naver.com/p/newssearch/3/api/tab/more"
        response = requests.get(url, headers=headers, cookies=cookies, params=params)

        print(f"[DEBUG] start={start}, status_code={response.status_code}")

        if response.status_code != 200:
            continue

        try:
            data = response.json()
            items = data.get("items", [])
            print(f"[DEBUG] {len(items)} items received")

            for item in items:
                news_data = {
                    "title": item.get("title", "").replace("<b>", "").replace("</b>", ""),
                    "summary": item.get("summary", ""),
                    "url": item.get("url", ""),
                    "date": item.get("date", ""),
                    "office": item.get("office", ""),
                }
                news_list.append(news_data)

        except Exception as e:
            print(f"[ERROR] JSON decode failed: {e}")
            print(response.text[:300])

        time.sleep(0.5)

    return news_list[:total]

# 사용
news_articles = crawl_naver_news()
print(f"총 수집된 기사 수: {len(news_articles)}")

for i, article in enumerate(news_articles, 1):
    print(f"[{i}] {article['title']}")
    print(f"    - URL: {article['url']}")
    print(f"    - 요약: {article['summary']}\n")


[DEBUG] start=1, status_code=200
[DEBUG] 0 items received
[DEBUG] start=11, status_code=200
[DEBUG] 0 items received
[DEBUG] start=21, status_code=200
[DEBUG] 0 items received
[DEBUG] start=31, status_code=200
[DEBUG] 0 items received
[DEBUG] start=41, status_code=200
[DEBUG] 0 items received
[DEBUG] start=51, status_code=200
[DEBUG] 0 items received
[DEBUG] start=61, status_code=200
[DEBUG] 0 items received
[DEBUG] start=71, status_code=200
[DEBUG] 0 items received
[DEBUG] start=81, status_code=200
[DEBUG] 0 items received
[DEBUG] start=91, status_code=200
[DEBUG] 0 items received
총 수집된 기사 수: 0
